In [1]:
from pathlib import Path

from skimage import morphology, exposure, filters
import numpy as np

from cytomancer.experiment import ExperimentType, Axes
from cytomancer.utils import load_experiment

path = Path("/nfs/bignasty/experiments/EKZ149")
experiment_type = ExperimentType.CQ1

experiment = load_experiment(path, experiment_type)

stack = experiment.isel({Axes.REGION: 0, Axes.FIELD: 0}).sel({Axes.CHANNEL: "DAPI"}).values

def preprocess(stack):
    eqd = np.array([exposure.equalize_adapthist(im, clip_limit=0.01) for im in stack])
    rescaled = np.array([exposure.rescale_intensity(im, out_range="uint8") for im in eqd])
    med = np.array([filters.rank.median(im, morphology.disk(5)) for im in rescaled])
    rescaled = np.array([exposure.rescale_intensity(im, out_range="float16") for im in med])
    return rescaled

def pred(model, stack):
    for im in stack:
        model.predict_instances(im)

eqd = preprocess(stack)


In [2]:
""" CPU - restart kernel if testing after GPU """
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
tf.config.threading.set_intra_op_parallelism_threads(4)
from stardist.models import StarDist2D, Config2D

model = StarDist2D().from_pretrained("2D_versatile_fluo")
model.predict_instances(eqd[0])

%time pred(model, eqd)

2024-05-10 12:36:51.088503: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-10 12:36:51.105535: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-10 12:36:51.105557: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-10 12:36:51.106330: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-10 12:36:51.109812: I tensorflow/core/platform/cpu_feature_guar

Using default values: prob_thresh=0.5, nms_thresh=0.4.
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
CPU times: user 20 s, sys: 189 ms, total: 20.2 s
Wall time: 5.04 s
